In [7]:
import requests
session = requests.session()
headers={"User-Agent": "Dalvik/2.1.0 (Linux; U; Android 5.1; PRO 5 Build/LMY47D)", 'accept-language': 'en'}


def authenticate():

    url = 'https://users.premierleague.com/accounts/login/'
    payload = {
    'password': '@Fantasy2024',
    'login': 'oyomworld@gmail.com',
    'redirect_uri': 'https://fantasy.premierleague.com/a/login',
    'app': 'plfpl-web'
    }
    response=session.post(url, data=payload,headers=headers)
    if response.status_code ==200:
        selected_player = session.get('https://fantasy.premierleague.com/api/my-team/5037001')
        print(selected_player)

        return selected_player.json()


picks=authenticate()
picks


<Response [200]>


{'picks': [{'element': 201,
   'position': 1,
   'selling_price': 45,
   'multiplier': 1,
   'purchase_price': 45,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 425,
   'position': 2,
   'selling_price': 45,
   'multiplier': 1,
   'purchase_price': 45,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 350,
   'position': 3,
   'selling_price': 60,
   'multiplier': 1,
   'purchase_price': 60,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 311,
   'position': 4,
   'selling_price': 70,
   'multiplier': 1,
   'purchase_price': 70,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 434,
   'position': 5,
   'selling_price': 55,
   'multiplier': 1,
   'purchase_price': 55,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 317,
   'position': 6,
   'selling_price': 75,
   'multiplier': 1,
   'purchase_price': 75,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 328,
   'position

In [6]:
selected_player = session.get('https://fantasy.premierleague.com/api/my-team/4037011')
selected_player

<Response [404]>

In [11]:
picks['transfers']

{'cost': 4, 'status': 'cost', 'limit': 1, 'made': 0, 'bank': 0, 'value': 1000}

In [12]:
response = session.get('https://fantasy.premierleague.com/api/me')
print(response.text)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [14]:
def  map_selected_picks(picks,players):
        picked_players = []
        for pick in picks:
            # Find the player with the matching id
            player = next((player for player in players if player["id"] == pick["element"]), None)
            if player:
                picked_players.append(player)

        return picked_players

In [15]:
all_players=session.get('https://fantasy.premierleague.com/api/bootstrap-static/')
all_players=all_players.json()

In [16]:
current_team=map_selected_picks(picks=picks['picks'],players=all_players['elements'])

In [11]:
def select_captain(players):
    def calculate_captain_score(player):
        # Criteria 1: Form (weight 30%)
        form_score = float(player['form']) * 0.3

        # Criteria 2: Fixture Difficulty (weight 25%)
        fixture_difficulty_score = 1 / (player['opponent_difficulty']) * 0.25 if 'opponent_difficulty' in player else 0

        # Criteria 3: Expected Minutes (weight 20%)
        minutes_score = (player['minutes'] / 90) * 0.2

        # Criteria 4: Set Piece Duties (weight 10%)
        set_piece_score = 0.1 if player['penalties_order'] else 0

        # Criteria 5: ICT Index (weight 10%)
        ict_index_score = float(player['ict_index']) * 0.2

        # Total Captain Score
        total_score = form_score + fixture_difficulty_score + minutes_score + set_piece_score + ict_index_score
        return total_score

    # Iterate through players and calculate their captain score
    captain_scores = []
    for player in players:
        score = calculate_captain_score(player)
        captain_scores.append((player['web_name'], score))

    # Select player with the highest score
    captain_scores.sort(key=lambda x: x[1], reverse=True)
    best_captain = captain_scores[:3]

    return best_captain

In [12]:
best_captain = select_captain(current_team)
print(best_captain)

[('Muniz', 0.3088888888888889), ('M.Salah', 0.1), ('Isak', 0.1)]


In [13]:
def suggest_transfers_out(players, budget, max_transfers=7):
    transfers_out = []

    for player in players:
        # Criteria 1: Low Form
        if float(player['form']) < 2.0:
            transfers_out.append(player)
            continue

        # Criteria 2: Injury or Suspension
        if player['status'] in ['d', 'i', 's']:
            transfers_out.append(player)
            continue

        # Criteria 3: Lack of Minutes
        if player['minutes'] < 90:
            transfers_out.append(player)
            continue

        # Criteria 4: Fixture Difficulty
        if 'upcoming_fixtures' in player:
            avg_difficulty = sum(player['upcoming_fixtures']) / len(player['upcoming_fixtures'])
            if avg_difficulty > 3.5:  # Assume difficulty is rated 1-5
                transfers_out.append(player)
                continue

        # Criteria 5: Ownership and Value
        if float(player['selected_by_percent']) < 5.0 and player['now_cost'] > 70:
            transfers_out.append(player)

    # Ensure we don't exceed the maximum number of transfers
    transfers_out = sorted(transfers_out, key=lambda x: float(x['form']))[:max_transfers]

    return transfers_out


# Suggest players to transfer out
budget = 1000  # Example budget
transfers_out = suggest_transfers_out(current_team, budget)

print("Players to consider transferring out:")
for player in transfers_out:
    print(f"{player['web_name']} - Form: {player['form']}, Status: {player['status']}, Minutes: {player['minutes']}, Cost: {player['now_cost']}")


Players to consider transferring out:
Pickford - Form: 0.0, Status: a, Minutes: 0, Cost: 50
Mykolenko - Form: 0.0, Status: a, Minutes: 0, Cost: 45
Gvardiol - Form: 0.0, Status: a, Minutes: 0, Cost: 60
Alexander-Arnold - Form: 0.0, Status: a, Minutes: 0, Cost: 70
Hudson-Odoi - Form: 0.0, Status: a, Minutes: 0, Cost: 55
Sancho - Form: 0.0, Status: a, Minutes: 0, Cost: 65
M.Salah - Form: 0.0, Status: a, Minutes: 0, Cost: 125


In [14]:
def identify_transfer_out(picks, players):
    transfer_out_candidates = []

    for pick in picks:
        player = next((p for p in players if p['id'] == pick['element']), None)

        if player:
            # Criteria to identify players to transfer out
            if float(player['form']) < 2.0 or player['status'] != 'a' or sum(player['upcoming_fixtures']) > 10:
                transfer_out_candidates.append(player)

    return transfer_out_candidates

def suggest_replacements(players_out, available_players, budget_remaining):
    replacements = []

    for player_out in players_out:
        # Filter available players by position
        possible_replacements = [p for p in available_players if p['element_type'] == player_out['element_type']]

        # Further filter by affordability
        possible_replacements = [p for p in possible_replacements if p['now_cost'] <= budget_remaining + player_out['now_cost']]

        # Sort by form, then by fixture difficulty
        possible_replacements = sorted(possible_replacements, key=lambda p: (-float(p['form']), sum(p['upcoming_fixtures'])))

        # Pick the top candidate
        if possible_replacements:
            replacements.append(possible_replacements[0])

    return replacements

def transfer_suggestions(picks, players, available_players, budget_remaining):
    # Identify players to transfer out
    players_out = identify_transfer_out(picks, players)

    # Suggest replacements for those players
    replacements = suggest_replacements(players_out, available_players, budget_remaining)

    return list(zip(players_out, replacements))

# Example data
picks = [
    {'element': 1, 'position': 1, 'selling_price': 50, 'multiplier': 1, 'purchase_price': 50, 'is_captain': False, 'is_vice_captain': False},
    {'element': 2, 'position': 2, 'selling_price': 45, 'multiplier': 1, 'purchase_price': 45, 'is_captain': False, 'is_vice_captain': False},
]

players = [
    {
        "id": 1,
        "element_type": 3,
        "web_name": "Fábio Vieira",
        "form": "0.0",
        "now_cost": 55,
        "status": "d",
        "upcoming_fixtures": [4, 4, 5]
    },
    {
        "id": 2,
        "element_type": 4,
        "web_name": "G. Jesus",
        "form": "0.0",
        "now_cost": 70,
        "status": "a",
        "upcoming_fixtures": [4, 4, 5]
    }
]

available_players = [
    {"web_name": "Saka", "form": "9.0", "element_type": 3, "now_cost": 80, "upcoming_fixtures": [2, 2, 2]},
    {"web_name": "Martinelli", "form": "7.0", "element_type": 3, "now_cost": 75, "upcoming_fixtures": [2, 3, 3]},
    {"web_name": "Maddison", "form": "8.5", "element_type": 3, "now_cost": 85, "upcoming_fixtures": [3, 3, 2]},
    {"web_name": "Wilson", "form": "7.0", "element_type": 4, "now_cost": 75, "upcoming_fixtures": [2, 4, 2]},
    {"web_name": "Isak", "form": "8.0", "element_type": 4, "now_cost": 70, "upcoming_fixtures": [2, 2, 1]},
    {"web_name": "Son", "form": "10.0", "element_type": 3, "now_cost": 100, "upcoming_fixtures": [2, 2, 2]},
]

# Budget remaining after transfers out (example)
budget_remaining = 100  # Example budget remaining

# Get transfer suggestions
suggestions = transfer_suggestions(picks, players, available_players, budget_remaining)

print("Transfer Suggestions:")
for player_out, replacement in suggestions:
    print(f"Transfer Out: {player_out['web_name']} (Form: {player_out['form']}, Cost: {player_out['now_cost']})")
    print(f"Replacement: {replacement['web_name']} (Form: {replacement['form']}, Cost: {replacement['now_cost']})")
    print("---")


Transfer Suggestions:
Transfer Out: Fábio Vieira (Form: 0.0, Cost: 55)
Replacement: Son (Form: 10.0, Cost: 100)
---
Transfer Out: G. Jesus (Form: 0.0, Cost: 70)
Replacement: Isak (Form: 8.0, Cost: 70)
---


In [15]:
def identify_transfer_out(picks, players):
    transfer_out_candidates = []

    for pick in picks:
        player = next((p for p in players if p['id'] == pick['element']), None)

        if player:
            # Criteria to identify players to transfer out
            if float(player['form']) < 2.0 or player['status'] != 'a' or sum(player['upcoming_fixtures']) > 10:
                transfer_out_candidates.append(player)

    return transfer_out_candidates

def suggest_replacements(players_out, available_players, budget_remaining):
    replacements = []

    for player_out in players_out:
        # Filter available players by position
        possible_replacements = [p for p in available_players if p['element_type'] == player_out['element_type']]

        # Further filter by affordability
        affordable_replacements = [p for p in possible_replacements if p['now_cost'] <= budget_remaining + player_out['now_cost']]

        # Sort by form, then by fixture difficulty
        affordable_replacements = sorted(affordable_replacements, key=lambda p: (-float(p['form']), sum(p['upcoming_fixtures'])))

        # Pick the top candidate
        if affordable_replacements:
            best_replacement = affordable_replacements[0]
            budget_remaining += player_out['now_cost'] - best_replacement['now_cost']
            replacements.append((player_out, best_replacement, budget_remaining))

    return replacements, budget_remaining

def transfer_suggestions(picks, players, available_players, initial_budget):
    budget_remaining = initial_budget

    # Identify players to transfer out
    players_out = identify_transfer_out(picks, players)

    # Suggest replacements for those players and update the budget
    replacements, final_budget = suggest_replacements(players_out, available_players, budget_remaining)

    return replacements, final_budget

# Example data
picks = [
    {'element': 1, 'position': 1, 'selling_price': 50, 'multiplier': 1, 'purchase_price': 50, 'is_captain': False, 'is_vice_captain': False},
    {'element': 2, 'position': 2, 'selling_price': 45, 'multiplier': 1, 'purchase_price': 45, 'is_captain': False, 'is_vice_captain': False},
]

players = [
    {
        "id": 1,
        "element_type": 3,
        "web_name": "Fábio Vieira",
        "form": "0.0",
        "now_cost": 55,
        "status": "d",
        "upcoming_fixtures": [4, 4, 5]
    },
    {
        "id": 2,
        "element_type": 4,
        "web_name": "G. Jesus",
        "form": "0.0",
        "now_cost": 70,
        "status": "a",
        "upcoming_fixtures": [4, 4, 5]
    }
]

available_players = [
    {"web_name": "Saka", "form": "9.0", "element_type": 3, "now_cost": 80, "upcoming_fixtures": [2, 2, 2]},
    {"web_name": "Martinelli", "form": "7.0", "element_type": 3, "now_cost": 75, "upcoming_fixtures": [2, 3, 3]},
    {"web_name": "Maddison", "form": "8.5", "element_type": 3, "now_cost": 85, "upcoming_fixtures": [3, 3, 2]},
    {"web_name": "Wilson", "form": "7.0", "element_type": 4, "now_cost": 75, "upcoming_fixtures": [2, 4, 2]},
    {"web_name": "Isak", "form": "8.0", "element_type": 4, "now_cost": 70, "upcoming_fixtures": [2, 2, 1]},
    {"web_name": "Son", "form": "10.0", "element_type": 3, "now_cost": 100, "upcoming_fixtures": [2, 2, 2]},
]

# Initial budget remaining after transfers out
initial_budget = 100  # Example budget remaining

# Get transfer suggestions
suggestions, final_budget = transfer_suggestions(picks, players, available_players, initial_budget)

print("Transfer Suggestions:")
for player_out, replacement, budget in suggestions:
    print(f"Transfer Out: {player_out['web_name']} (Form: {player_out['form']}, Cost: {player_out['now_cost']})")
    print(f"Replacement: {replacement['web_name']} (Form: {replacement['form']}, Cost: {replacement['now_cost']})")
    print(f"Remaining Budget After Transfer: {budget}")
    print("---")

print(f"Final Budget After All Transfers: {final_budget}")


Transfer Suggestions:
Transfer Out: Fábio Vieira (Form: 0.0, Cost: 55)
Replacement: Son (Form: 10.0, Cost: 100)
Remaining Budget After Transfer: 55
---
Transfer Out: G. Jesus (Form: 0.0, Cost: 70)
Replacement: Isak (Form: 8.0, Cost: 70)
Remaining Budget After Transfer: 55
---
Final Budget After All Transfers: 55
